# Scrapping Ontario Legislative data

We want to go after
* Voting Record of each MPP
* Bills they've Sponsored 
* Committees they're on

## Important Links
List of MPPs

> http://www.ontla.on.ca/web/members/members_current.do?locale=en

List of all bills and their sponsors back to 1995 

> http://www.ontla.on.ca/web/bills/bills_all.do?locale=en

Hansards back to 1975 

> http://www.ontla.on.ca/pas/house-proceedings/house-documents.xhtml?locale=en&parl=&sess=&year=&month=



In [1]:
import pandas as pd
import numpy as np
import urllib2

from bs4 import BeautifulSoup


In [2]:
url_main = 'http://www.ontla.on.ca/web/bills/bills_all.do?locale=en'

# query the website and return the html to the variable ‘page’
html = urllib2.urlopen(url_main).read()

# parse the html using beautiful soap and store in variable `soup`
soup = BeautifulSoup(html, 'html.parser')



In [3]:
# Follow Links on the page to get all bill pages
# Par41Sess3 = http://www.ontla.on.ca/web/bills/bills_all.do?locale=en&parlSessionID=41%3A1&go=go
    
#url for the first page of bills
url_main = 'http://www.ontla.on.ca/web/bills/bills_all.do?locale=en'

#For creating link IDs on the fly
url_stub = 'http://www.ontla.on.ca/web/bills/bills_all.do?locale=en&parlSessionID='

html = urllib2.urlopen(url_main).read()
soup = BeautifulSoup(html, 'html.parser')

#Create Links for all available bill page
Sessions = soup.find_all('option')

# tmp = [i.text.split(':') for i in Sessions]

# Sess = [i[0] for i in tmp]
Sess = [str(i.text.split(':')[0]).strip('\n') for i in Sessions]
Sitting = [ str(i.text.split(':')[1].split('\n')[0]) for i in Sessions]

#Links to all Bill pages
urls = [url_stub + str(Sess[i]) + '%3A' + str(Sitting[i]) + '&go=go' for i in range(len(Sess))]



In [4]:
BillID = []
BillNames = []
SponsorNames = []
BillNumber = []
BillURL = []


for i in range(len(urls)):
    html = urllib2.urlopen(urls[i]).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    filename = 'HtmlBillsAndSponsors/Parliament-' + str(Sess[i]) + '-Session-' + str(Sitting[i]) + '.html'
    Html_file= open(filename,"w")
    Html_file.write(html)
    Html_file.close()
    
    tempBillNames = soup.find_all('td',attrs={'class':'titlecell'})
    tempSponsorNames = soup.find_all('td',attrs={'class':'sponsorcell'})
    tempBillNumber = soup.find_all('td', attrs={'class': 'nocell'})
    BillNames = BillNames + tempBillNames
    SponsorNames = SponsorNames + tempSponsorNames
    BillNumber = BillNumber + tempBillNumber

In [5]:
# print(soup.prettify())
len(urls)

15

In [88]:
dfBills = pd.DataFrame(index = range(len(BillNames)), columns = ('ID','Number','Name', 'Sponsor','URL'))

#Need to strip characters out of the fields (lots of \n and other tags)

for i in range(len(BillNames)):
#     print BillNames[i].text
#     dfBills.iloc[i] = [BillNames[i].text, SponsorNames[i].text]
    dfBills['Name'].iloc[i] = (BillNames[i].text.strip('\n').replace(u'\xa0', ''))
    dfBills['Sponsor'].iloc[i] =(SponsorNames[i].text
                                 .strip(u'\n\n\n\xa0\n\n')
                                 .replace('\n\n\n','; ')
                                 .replace('\n','')
                                 .replace(u'\xa0','')
                                )
    dfBills['Number'].iloc[i] = (BillNumber[i].text
                                 .strip('\n')
                                 .split(' ')[0]
                                 .replace(u'\xa0;','')
                                )
    
    #Strip out Bill IDs that they use for the electronic tracking
    ref = BillNames[i].find('a').get('href')
    strtTxt = 'BillID='
    endTxt = '&'
    strt = ref.index(strtTxt) + len(strtTxt)
    end = ref.index(endTxt,strt) 
    BillID = ref[strt:end]


    dfBills['ID'].iloc[i]= BillID    
    dfBills['URL'].iloc[i]= 'http://www.ontla.on.ca/web/bills/bills_detail.do?locale=en&BillID='+ BillID +'&isCurrent=false&ParlSessionID='


# dir(dfBills)

In [86]:
# BillNumber[2].text
# SponsorNames[0].text.split(u'\xa0')
# SponsorNames
ref = BillNames[1].find('a').get('href')

strtTxt = 'BillID='
endTxt = '&'
strt = ref.index(strtTxt) + len(strtTxt)
end = ref.index(endTxt,strt) 
BillID = ref[strt:end]
print BillID

# urlBill = 'http://www.ontla.on.ca/web/bills/bills_detail.do?locale=en&BillID='+ BillID +'&isCurrent=false&ParlSessionID='
# print urlBill

4113


In [90]:
dfBills.iloc[2000:2050]


,ID,Number,Name,Sponsor,URL
2000,966,28,"Human Rights Code Amendment Act, 2001","Ramsay, David",http://www.ontla.on.ca/web/bills/bills_detail....
2001,968,29,"Oak Ridges Moraine Green Planning Act, 2001","Churley, Marilyn",http://www.ontla.on.ca/web/bills/bills_detail....
2002,1199,30,Volunteer Firefighters Employment Protection A...,"Arnott, Ted",http://www.ontla.on.ca/web/bills/bills_detail....
2003,975,31,"Labour Relations Amendment Act, 2001","Martel, Shelley",http://www.ontla.on.ca/web/bills/bills_detail....
2004,976,32,"Petroleum Products Price Freeze Act, 2001","Bartolucci, Rick",http://www.ontla.on.ca/web/bills/bills_detail....
2005,978,33,Jay Lawrence and Bart Mackey Memorial Act (Hig...,"Galt, Doug",http://www.ontla.on.ca/web/bills/bills_detail....
2006,984,34,Dave Ellis Memorial Occupational Health and Sa...,"Agostino, Dominic",http://www.ontla.on.ca/web/bills/bills_detail....
2007,986,35,"Irish Heritage Day Act, 2001","O'Toole, John",http://www.ontla.on.ca/web/bills/bills_detail....
2008,987,36,"Truth About Ipperwash Act, 2001","Phillips, Gerry",http://www.ontla.on.ca/web/bills/bills_detail....
2009,988,37,"Affordable Housing Incentives Act, 2001","Caplan, David",http://www.ontla.on.ca/web/bills/bills_detail....


In [91]:
#Output the Data Frame to a CSV

dfBills.to_excel('2017-11-10-Bills-Sponsors-IDS-URLs.xls')